In [13]:
import pandas as pd
import numpy as np

In [14]:
df_train = pd.read_csv('data/instacart/train_user2.csv', sep=',')
df_test = pd.read_csv('data/instacart/test_user2.csv', sep=',')

In [15]:
df_train = df_train.sort_values('order_id')
df_train = df_train[['order_id', 'aisle_id']].drop_duplicates()
df_train

,order_id,aisle_id
29206,3,91
24807,3,120
72983,3,83
82088,3,112
36932,3,123
...,...,...
1503847,3421072,37
905057,3421072,24
679170,3421072,83
2625779,3421073,104


In [16]:
df_test = df_test.sort_values('order_id')
df_test = df_test[['order_id', 'aisle_id']].drop_duplicates()
df_test

,order_id,aisle_id
147709,21,24
147896,21,26
147650,21,14
147859,21,21
206821,44,34
...,...,...
491931,3421078,3
572173,3421078,67
88233,3421078,24
471571,3421078,121


In [17]:
df_train[:20]

,order_id,aisle_id
29206,3,91
24807,3,120
72983,3,83
82088,3,112
36932,3,123
79561,3,35
701514,35,107
700218,35,98
701931,35,61
700138,35,8


In [18]:
df_test[:20]

,order_id,aisle_id
147709,21,24
147896,21,26
147650,21,14
147859,21,21
206821,44,34
206278,44,121
206255,44,77
206244,44,52
206838,44,29
206857,44,67


In [19]:
print(df_test['aisle_id'].max(), df_train['aisle_id'].max())

134 134


Label for train test set

In [20]:
df_train['x_label'] = [0] * len(df_train)
df_test['x_label'] = [1] * len(df_test)

In [21]:
df_train.head()

,order_id,aisle_id,x_label
29206,3,91,0
24807,3,120,0
72983,3,83,0
82088,3,112,0
36932,3,123,0


In [22]:
df_test.head()

,order_id,aisle_id,x_label
147709,21,24,1
147896,21,26,1
147650,21,14,1
147859,21,21,1
206821,44,34,1


Concat 2 set

In [23]:
data = pd.concat([df_train, df_test]).sort_values('order_id')
data[:20]

,order_id,aisle_id,x_label
29206,3,91,0
24807,3,120,0
72983,3,83,0
82088,3,112,0
36932,3,123,0
79561,3,35,0
147650,21,14,1
147859,21,21,1
147896,21,26,1
147709,21,24,1


Reindex -> feed to graph order_id 0 -> n_order 
                         aisle_id 0 -> n_aisle (-=1)

**Construct item-item co-occurence edge from item_graph_dict.npy**

In [24]:
import numpy as np
import pandas as pd

In [25]:
i_i = np.load('data/instacart/item_graph_dict.npy', allow_pickle=True).item()
i_i[0]

FileNotFoundError: [Errno 2] No such file or directory: 'data/instacart/item_graph_dict.npy'

In [ ]:
i_i[0][0]

In [ ]:
len(i_i[4][0]) #4 vs rest \ {4} -> 133

133

In [ ]:
i_i[4][1] # it's sorted max -> min -> get top-10 from 

In [ ]:
i_i[0][0][:10]

[122, 82, 23, 20, 106, 119, 111, 83, 77, 115]

In [ ]:
def construct_top_k(i_i, k):
    i1 = []
    i2 = []
    i3 = []
    for i in i_i.keys():
        i1.extend([i] * k)
        i2.extend(i_i[i][0][:k])
        i3.extend(i_i[i][1][:k])
    return i1, i2, i3

In [ ]:
col1, col2, col3 = construct_top_k(i_i=i_i, k=10)
top_k_dict = {'cate_id': col1, 'top_k': col2, 'weight': col3}
df = pd.DataFrame(top_k_dict)
df

,cate_id,top_k,weight
0,0,122,1736.0
1,0,82,1712.0
2,0,23,1696.0
3,0,20,1695.0
4,0,106,1616.0
...,...,...,...
1335,133,106,246.0
1336,133,111,246.0
1337,133,107,229.0
1338,133,115,227.0


In [ ]:
df['x_label'] = [0] * len(df)

In [ ]:
df.to_csv('data/instacart/item_item.csv', sep='\t', index=False)

**Construct user-item**

In [ ]:
u_i = pd.read_csv('data/instacart/interaction.csv', sep='\t')
u_i

,u_id,aisle_id,x_label
0,0,114,0
1,0,130,0
2,0,82,0
3,0,35,0
4,0,23,0
...,...,...,...
537774,8024,57,2
537775,8024,6,2
537776,8024,36,2
537777,8024,63,2


In [ ]:
u_i[u_i['u_id'] == 0]

,u_id,aisle_id,x_label
0,0,114,0
1,0,130,0
2,0,82,0
3,0,35,0
4,0,23,0
...,...,...,...
58,0,36,1
59,0,48,1
60,0,115,2
61,0,90,2


In [ ]:
user_id = u_i['u_id'].to_numpy()
item_id = u_i['aisle_id'].to_numpy()

In [ ]:
user_item_dict = {}
l = len(u_i)

for i in range(0, 8025):
    user_item_dict[i] = []

for i in range(0, l):
    user_item_dict[user_id[i]].append(item_id[i])

In [ ]:
len(user_item_dict[0])

63

In [ ]:
np.save('data/instacart/user_item_dict.npy', user_item_dict)